In [ ]:
import marimo as mo

# Week 6: Regularization and Generalization**IME775: Data Driven Modeling and Optimization**📖 **Reference**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 7---## Learning Objectives- Understand the bias-variance tradeoff- Master L2 regularization and dropout- Learn batch normalization- Implement early stopping

In [ ]:
import numpy as npimport matplotlib.pyplot as pltfrom sklearn.model_selection import train_test_split

## 6.1 The Overfitting Problem**Overfitting**: Model memorizes training data but fails on new data.- Training loss ↓ but validation loss ↑- High variance, low bias

## 6.2 L2 Regularization (Weight Decay)Add penalty on weight magnitudes:$$L_{total} = L_{data} + \frac{\lambda}{2} \|W\|^2$$**Effect**: Shrinks weights, reduces model complexity

## 6.3 DropoutDuring training, randomly zero out neurons with probability $p$:$$h = \frac{1}{1-p} \cdot \text{mask} \odot \sigma(z)$$**Effect**: Creates implicit ensemble of sub-networks

In [ ]:
# Demonstrate dropoutclass DropoutDemo:        self.p = p        self.training = True        if self.training:            mask = (np.random.rand(*x.shape) > self.p)            return x * mask / (1 - self.p)        return x# Visualize dropout effectnp.random.seed(42)x = np.ones((1, 10))  # 10 neurons, all activefig3, axes3 = plt.subplots(2, 4, figsize=(14, 6))dropout = DropoutDemo(p=0.5)# Show 8 different dropout masksfor i, ax in enumerate(axes3.flat):    np.random.seed(i)    output = dropout.forward(x.copy())    colors = ['green' if v > 0 else 'red' for v in output[0]]    ax.bar(range(10), output[0], color=colors)    ax.axhline(1, color='blue', linestyle='--', alpha=0.5)    ax.set_ylim(0, 2.5)    ax.set_title(f'Sample {i+1}: {int(np.sum(output > 0))}/10 active')    ax.set_xticks(range(10))    if i >= 4:        ax.set_xlabel('Neuron')    if i % 4 == 0:        ax.set_ylabel('Output')plt.suptitle('Dropout (p=0.5): Different Random Masks\nGreen=Active (scaled by 2), Red=Dropped', fontsize=12, y=1.02)plt.tight_layout()fig3

## 6.4 Batch NormalizationNormalize activations within each mini-batch:$$\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$$$y = \gamma \hat{x} + \beta$$**Benefits**: Faster training, regularization, higher learning rates

## 6.5 Early StoppingStop training when validation loss starts increasing:1. Monitor validation loss2. Save best model3. Stop if no improvement for $k$ epochs

## 6.6 Data AugmentationArtificially expand training data by applying transformations.| Type | Transformations ||------|-----------------|| Geometric | Rotation, flip, crop, scale || Color | Brightness, contrast, hue || Noise | Gaussian noise, blur || Mixing | Mixup, CutMix |

## 6.7 Mixup: Advanced AugmentationCreate virtual training examples by mixing:$$\tilde{x} = \lambda x_i + (1-\lambda) x_j$$$$\tilde{y} = \lambda y_i + (1-\lambda) y_j$$Where $\lambda \sim \text{Beta}(\alpha, \alpha)$

In [ ]:
# Demonstrate Mixup    np.random.seed(42)    # Two sample "images" (simplified as 1D signals)    x1 = np.sin(np.linspace(0, 2*np.pi, 50))    x2 = np.cos(np.linspace(0, 2*np.pi, 50))    y1, y2 = np.array([1, 0]), np.array([0, 1])  # One-hot labels    # Different lambda values    lambdas_mix = [0.0, 0.3, 0.5, 0.7, 1.0]    fig, axes = plt.subplots(1, 5, figsize=(15, 3))    for ax, lam in zip(axes, lambdas_mix):        x_mixed = lam * x1 + (1 - lam) * x2        y_mixed = lam * y1 + (1 - lam) * y2        ax.plot(x_mixed, 'purple', linewidth=2)        ax.set_title(f'λ={lam}\ny=[{y_mixed[0]:.1f}, {y_mixed[1]:.1f}]')        ax.set_ylim(-1.5, 1.5)    plt.suptitle('Mixup: Interpolating Between Two Samples', fontsize=12, y=1.05)    plt.tight_layout()    return figfig7 = visualize_mixup()fig7

## Summary| Technique | Mechanism | When to Use ||-----------|-----------|-------------|| **L2 Regularization** | Penalize large weights | Always (default) || **Dropout** | Random neuron dropping | Fully connected layers || **BatchNorm** | Normalize activations | CNNs || **Early Stopping** | Stop at best validation | Always || **Data Augmentation** | Transform training data | Limited data |---## References- **Primary**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 7.- **Dropout**: Srivastava et al. (2014). "Dropout: A simple way to prevent overfitting."- **BatchNorm**: Ioffe & Szegedy (2015). "Batch Normalization."## Connection to ML Refined CurriculumRegularization prevents overfitting for:- All supervised learning (Weeks 4-8)- Feature selection (Week 9)